In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json

import sys

sys.path.append("../")

##################################################################
os.environ["TOKENIZERS_PARALLELISM"] = "false"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3,4,5,6,7"
##################################################################

import logging
from src.utils import logging_utils
from src.utils import env_utils

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

import torch
import transformers

logger.info(f"{torch.__version__=}, {torch.version.cuda=}")
logger.info(
    f"{torch.cuda.is_available()=}, {torch.cuda.device_count()=}, {torch.cuda.get_device_name()=}"
)
logger.info(f"{transformers.__version__=}")

/share/u/arnab/miniconda3/envs/connection/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-09-10 13:51:15 __main__ INFO     torch.__version__='2.8.0+cu126', torch.version.cuda='12.6'
2025-09-10 13:51:15 __main__ INFO     torch.cuda.is_available()=True, torch.cuda.device_count()=1, torch.cuda.get_device_name()='NVIDIA RTX A6000'
2025-09-10 13:51:15 __main__ INFO     transformers.__version__='4.56.1'


In [3]:
from src.utils.training_utils import get_device_map

# model_key = "meta-llama/Llama-3.2-3B"
# model_key = "meta-llama/Llama-3.1-8B"
# model_key = "meta-llama/Llama-3.1-70B-Instruct"
# model_key = "meta-llama/Llama-3.3-70B-Instruct"
# model_key = "meta-llama/Llama-3.1-405B-Instruct"

# model_key = "google/gemma-2-9b-it"
# model_key = "google/gemma-3-12b-it"
model_key = "google/gemma-2-27b-it"

# model_key = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

# model_key = "allenai/OLMo-2-1124-7B-Instruct"
# model_key = "allenai/OLMo-7B-0424-hf"

# model_key = "Qwen/Qwen2-7B"
# model_key = "Qwen/Qwen2.5-14B-Instruct"
# model_key = "Qwen/Qwen2.5-32B-Instruct"
# model_key = "Qwen/Qwen2.5-72B-Instruct"

# model_key = "Qwen/Qwen3-1.7B"
# model_key = "Qwen/Qwen3-4B"
# model_key = "Qwen/Qwen3-8B"
# model_key = "Qwen/Qwen3-14B"
# model_key = "Qwen/Qwen3-32B"

# device_map = get_device_map(model_key, 30, n_gpus=8)
# device_map

2025-09-10 13:51:18 git.cmd DEBUG    Popen(['git', 'version'], cwd=/share/u/arnab/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)
2025-09-10 13:51:18 git.cmd DEBUG    Popen(['git', 'version'], cwd=/share/u/arnab/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)
2025-09-10 13:51:18 wandb.docker.auth DEBUG    Trying paths: ['/share/u/arnab/.docker/config.json', '/share/u/arnab/.dockercfg']
2025-09-10 13:51:18 wandb.docker.auth DEBUG    No config file found


In [4]:
from src.models import ModelandTokenizer

# from transformers import BitsAndBytesConfig

mt = ModelandTokenizer(
    model_key=model_key,
    torch_dtype=torch.bfloat16,
    # device_map=device_map,
    device_map="auto",
    # quantization_config = BitsAndBytesConfig(
    #     # load_in_4bit=True
    #     load_in_8bit=True
    # )
    attn_implementation="eager",
)

2025-09-10 13:51:19 src.models WARNING  google/gemma-2-27b-it not found in /share/u/models
If not found in cache, model will be downloaded from HuggingFace to cache directory
2025-09-10 13:51:19 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443
2025-09-10 13:51:19 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /google/gemma-2-27b-it/resolve/main/config.json HTTP/1.1" 200 0


`torch_dtype` is deprecated! Use `dtype` instead!


2025-09-10 13:51:20 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /google/gemma-2-27b-it/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
2025-09-10 13:51:20 urllib3.connectionpool DEBUG    https://huggingface.co:443 "GET /api/models/google/gemma-2-27b-it/tree/main/additional_chat_templates?recursive=False&expand=False HTTP/1.1" 404 64


`torch_dtype` is deprecated! Use `dtype` instead!


2025-09-10 13:51:20 accelerate.utils.modeling INFO     We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards: 100%|██████████| 12/12 [00:06<00:00,  1.91it/s]

2025-09-10 13:51:27 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /google/gemma-2-27b-it/resolve/main/generation_config.json HTTP/1.1" 200 0
2025-09-10 13:51:27 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /google/gemma-2-27b-it/resolve/main/custom_generate/generate.py HTTP/1.1" 404 0


2025-09-10 13:51:27 accelerate.big_modeling WARNING  Some parameters are on the meta device because they were offloaded to the cpu.
2025-09-10 13:51:27 src.models INFO     loaded model <google/gemma-2-27b-it> | size: 51931.626 MB | dtype: torch.bfloat16 | device: cuda:0


In [5]:
from src.selection.data import SelectOneTask, CountingTask, YesNoTask, SelectFirstTask
from src.selection.data import SelectionSample, CountingSample, YesNoSample

#################################################################################
# TASK_CLS = SelectOneTask
# TASK_CLS = CountingTask
# TASK_CLS = YesNoTask
TASK_CLS = SelectFirstTask
prompt_template_idx = 0
N_DISTRACTORS = 5
OPTION_STYLE = "single_line"
#################################################################################

select_task = TASK_CLS.load(
    path=os.path.join(
        env_utils.DEFAULT_DATA_DIR,
        "selection",
        # "profession.json"
        # "nationality.json"
        "objects.json",
    )
)

print(select_task)

{'name': 'different objects', 'prompt_templates': ['Which object from the following list shares its category with <_pivot_entity_>?\n<_options_>\nAnswer:', '<_options_>\nWhich among these objects mentioned above share the same category as <_pivot_entity_>?\nAnswer:', 'Which object from the following list is a <_category_>?\n<_options_>\nAnswer:', '<_options_>\nWhich among these objects mentioned above is a <_category_>?\nAnswer:'], 'odd_one_prompt_templates': ['Which object from the following list does not belong to the same category as <_pivot_entity_>?\n<_options_>\nAnswer:', '<_options_>\nWhich among these objects mentioned above does not belong to the same category as <_pivot_entity_>?\nAnswer:', 'Which object from the following list is not a <_category_>?\n<_options_>\nAnswer:', '<_options_>\nWhich among these objects mentioned above is not a <_category_>?\nAnswer:', 'Detect the odd one out from the following list of objects:\n<_options_>\nAnswer:', '<_options_>\nWhich object in t

In [15]:
from src.functional import predict_next_token, generate_with_patch

sample = select_task.get_random_sample(
    mt=mt, 
    category="fruit",
    filter_by_lm_prediction=True,
    prompt_template_idx=3,
    # n_options=5,
    n_distractors=3,
)
print(f'"{sample.prompt()}" >> {mt.tokenizer.decode(sample.ans_token_id)}')

gen = generate_with_patch(
    mt=mt, 
    inputs=sample.prompt(),
    n_gen_per_prompt=1,
    remove_prefix=True,
    max_new_tokens=20,
    do_sample=False,
)
print(f"Generation:\"{gen[0]}\"")

"Options: Scissors, Watermelon, Pineapple, Bed, Carrot, Strawberry, Apple.
What is the first fruit from the list above?
Answer:" >>  Watermelon
Generation:" Watermelon

Explanation:
The first fruit in the list is watermelon. 

Let me know if"


In [16]:
sample.prediction

[PredictedToken(token=' Watermelon', prob=0.97265625, logit=23.125, token_id=197201, metadata=None),
 PredictedToken(token=' Apple', prob=0.0157470703125, logit=19.0, token_id=9865, metadata=None),
 PredictedToken(token=' **', prob=0.007415771484375, logit=18.25, token_id=5231, metadata=None),
 PredictedToken(token='  ', prob=0.00113677978515625, logit=16.375, token_id=139, metadata=None),
 PredictedToken(token='\n\n', prob=0.000888824462890625, logit=16.125, token_id=109, metadata=None)]

In [19]:
from src.selection.data import get_counterfactual_samples_interface

counterfactual_sampler = get_counterfactual_samples_interface[select_task.task_name]

print(counterfactual_sampler)

patch_sample, clean_sample = counterfactual_sampler(
    mt=mt,
    task=select_task,
    patch_category="fruit",
    clean_category="vehicle",
    filter_by_lm_prediction=True,
    prompt_template_idx=3,
    option_style=OPTION_STYLE,
    # distinct_options=True,
    n_options=6
)

# patch_sample.default_option_style = "single_line"
# clean_sample.default_option_style = "numbered"

print("-" * 100)
print(patch_sample.prompt(), ">>", mt.tokenizer.decode(patch_sample.ans_token_id))
print(clean_sample.prompt(), ">>", mt.tokenizer.decode(clean_sample.ans_token_id))

<function get_counterfactual_samples_within_first_task at 0x76640102bce0>
----------------------------------------------------------------------------------------------------
Options: Airplane, Van, Pear, Hairdryer, Church, Plum.
What is the first fruit from the list above?
Answer: >>  Pear
Options: Boat, Motorcycle, Peach, Sink, Hospital, Orange.
What is the first vehicle from the list above?
Answer: >>  Boat


In [ ]:
patch_sample.prediction, clean_sample.prediction

(OrderedDict([(6287,
               (1,
                PredictedToken(token=' Yes', prob=0.99609375, logit=22.875, token_id=6287, metadata=None))),
              (1307,
               (13,
                PredictedToken(token=' No', prob=7.3909759521484375e-06, logit=11.0625, token_id=1307, metadata=None)))]),
 OrderedDict([(6287,
               (1,
                PredictedToken(token=' Yes', prob=0.99609375, logit=23.5, token_id=6287, metadata=None))),
              (1307,
               (8,
                PredictedToken(token=' No', prob=3.123283386230469e-05, logit=13.125, token_id=1307, metadata=None)))]))